In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hinge-english-to-hinglish-machine-translation/no_label.csv
/kaggle/input/hinge-english-to-hinglish-machine-translation/synthetic-dataset/valid.csv
/kaggle/input/hinge-english-to-hinglish-machine-translation/synthetic-dataset/train.csv
/kaggle/input/hinge-english-to-hinglish-machine-translation/synthetic-dataset/.ipynb_checkpoints/train-checkpoint.csv
/kaggle/input/hinge-english-to-hinglish-machine-translation/human-generated-dataset/valid_human_generated.pkl
/kaggle/input/hinge-english-to-hinglish-machine-translation/human-generated-dataset/train_human_generated.pkl
/kaggle/input/hinglish-top-dataset/Synthetically Generated Data/train.tsv
/kaggle/input/hinglish-top-dataset/Human Annotated Data/test.tsv
/kaggle/input/hinglish-top-dataset/Human Annotated Data/train.tsv
/kaggle/input/hinglish-top-dataset/Human Annotated Data/validation.tsv
/kaggle/input/language-translation-englishfrench/eng_-french.csv


In [2]:
import pandas as pd 
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
data=pd.read_csv('/kaggle/input/hinglish-top-dataset/Synthetically Generated Data/train.tsv',sep='\t')
data.head()

,en_query,cs_query,en_parses,cs_parses
0,Who is the featured performer at The Grand Ole...,Is Saturday night The Grand Ole Opry me featur...,[IN:GET_EVENT Who is [SL:CATEGORY_EVENT the fe...,[IN:GET_EVENT [SL:DATE_TIME Is Saturday night ...
1,Remind me of my meeting at 12 am,mujhe 12 am ko meri meeting ke baare me yaad d...,[IN:CREATE_REMINDER Remind [SL:PERSON_REMINDED...,[IN:CREATE_REMINDER [SL:PERSON_REMINDED mujhe ...
2,set an alarm for 5 am,subha 5 baje ke liye ek alarm set karen,[IN:CREATE_ALARM set an alarm [SL:DATE_TIME fo...,[IN:CREATE_ALARM [SL:DATE_TIME subha 5 baje ke...
3,What ’ s will weather be like at Sydney at 2 pm,dopaher 2 baje Sydney me mausam kaisa hoga,[IN:GET_WEATHER What ’s will weather be like a...,[IN:GET_WEATHER [SL:DATE_TIME dopaher 2 baje ]...
4,What is the weather in Canada ?,Canada me mausam kaisa hai ?,[IN:GET_WEATHER What is the weather in [SL:LOC...,[IN:GET_WEATHER [SL:LOCATION Canada ] me mausa...


In [6]:
data

,en_query,cs_query,en_parses,cs_parses
0,Who is the featured performer at The Grand Ole...,Is Saturday night The Grand Ole Opry me featur...,[IN:GET_EVENT Who is [SL:CATEGORY_EVENT the fe...,[IN:GET_EVENT [SL:DATE_TIME Is Saturday night ...
1,Remind me of my meeting at 12 am,mujhe 12 am ko meri meeting ke baare me yaad d...,[IN:CREATE_REMINDER Remind [SL:PERSON_REMINDED...,[IN:CREATE_REMINDER [SL:PERSON_REMINDED mujhe ...
2,set an alarm for 5 am,subha 5 baje ke liye ek alarm set karen,[IN:CREATE_ALARM set an alarm [SL:DATE_TIME fo...,[IN:CREATE_ALARM [SL:DATE_TIME subha 5 baje ke...
3,What ’ s will weather be like at Sydney at 2 pm,dopaher 2 baje Sydney me mausam kaisa hoga,[IN:GET_WEATHER What ’s will weather be like a...,[IN:GET_WEATHER [SL:DATE_TIME dopaher 2 baje ]...
4,What is the weather in Canada ?,Canada me mausam kaisa hai ?,[IN:GET_WEATHER What is the weather in [SL:LOC...,[IN:GET_WEATHER [SL:LOCATION Canada ] me mausa...
...,...,...,...,...
170078,How many miles from Humble to Brenham,Humble se Brenham tak kitne miles hai,[IN:GET_DISTANCE How many [SL:UNIT_DISTANCE mi...,[IN:GET_DISTANCE [SL:SOURCE Humble ] se [SL:DE...
170079,What is traffic so bad on Ronald Reagan,Ronald Reagan par traffic kyu kharaab hai,[IN:GET_INFO_TRAFFIC What is traffic so bad on...,[IN:GET_INFO_TRAFFIC [SL:LOCATION Ronald Reaga...
170080,Concerts by The Strokes for this weekend,Is weekend The Strokes ke hone wale Concerts,[IN:GET_EVENT [SL:CATEGORY_EVENT Concerts ] by...,[IN:GET_EVENT [SL:DATE_TIME Is weekend ] [SL:N...
170081,How many mm of rain do we expect for tomorrow ?,kal kitni mm baarish ki ummeed hai ?,[IN:UNSUPPORTED_WEATHER How many mm of [SL:WEA...,[IN:UNSUPPORTED_WEATHER kal kitni mm [SL:WEATH...


In [7]:
data=data[:50000]

In [8]:
SOS_token = 0
EOS_token = 1
UNK_token=2
class Lang:
    def __init__(self,name):
        self.name=name
        self.word2index={}
        self.word2count={}
        self.index2word={0:'sos',1:'eos','unk':2}
        self.n_words=2
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word]=self.n_words
            self.word2count[word]=1
            self.index2word[self.n_words]=word
            self.n_words+=1
        else:
            self.word2count[word]+=1
            

In [9]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [10]:
def readLangs(lines,lang1,lang2,reverse=False):
    print('Preparing Vocabulary.......')    
    pairs=[[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    if reverse:
        pairs=[list(reversed(p)) for p in pairs]
    
    input_lang=Lang(lang1)
    output_lang=Lang(lang2)
    
    return input_lang,output_lang,pairs


In [11]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
def filterPair(p):
    return len(p[0].split(' '))<MAX_LENGTH and len(p[1].split(' '))<MAX_LENGTH #and p[1].startswith(eng_prefixes)
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [12]:
def prepareData(data,lang1,lang2,reverse=False):
    print('Reading lines...')
    lines=[]
    for e_sent,f_sent in zip(data['cs_query'],data['en_query']):
        both_sent=e_sent+'\t'+f_sent
        lines.append(both_sent)
    input_lang,output_lang,pairs=readLangs(lines,lang1,lang2,reverse)
    print(f'Read {len(pairs)} sentence pairs')
    pairs=filterPairs(pairs)
    print(f'Trimmed to {len(pairs)} sentence pairs')
    print('Counting words')
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print('Counted words :')
    print(input_lang.name,input_lang.n_words)
    print(output_lang.name,output_lang.n_words)
    return input_lang,output_lang,pairs
input_lang,output_lang,pairs=prepareData(data,'eng','Hinglish',True)
print(random.choice(pairs))

Reading lines...
Preparing Vocabulary.......
Read 50000 sentence pairs
Trimmed to 27839 sentence pairs
Counting words
Counted words :
eng 5971
Hinglish 6393
['networking parties happening this weekend', 'is weekend hone wale networking parties']


# Model

In [13]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [14]:
class Decoder(nn.Module):
    
    def __init__(self,hidden_size,output_size):
        super(DecoderRNN,self).__init__()
        self.embedding = nn.Embedding(output_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.out = nn.Linear(hidden_size,output_size)
    
    def forward(self,encoder_outputs,encoder_hiddens,target_tensor=None):
        batch_size=encoder_output_size(0)
        decoder_input=torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(SOS_token)
        decoder_hidden=encoder_hidden
        decoder_output=[]
        
        for i in range(MAX_LENGTH):
            decoder_output,decoder_hidden=self.forward_step(decoder_input,decoder_hidden)
            decoder_output.append(decoder_output)
            
            if target_tensor is not None:
                decoder_input = target_tensor[:,i].unsqueeze(1)
            else:
                _,topi=decoder_output.topk(1)
                decoder_input=topi.squeeze(-1).detach()

# Attentions 

In [15]:
#attention 1.
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)
        
        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

    def forward_inference(self, query, keys,):
        print(20*'-',' Attention Inference start ',20*'-')
        print('Query : ',query.shape)
        print('Keys : ',keys.shape)
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        print('Scores : ',scores.shape)
        scores = scores.squeeze(2).unsqueeze(1)
        print('Scores squeeze(2) :',scores.squeeze(2).shape)
        print('scores.squeeze(2).unsqueeze(1) : ',scores.squeeze(2).unsqueeze(1).shape)
        
        weights = F.softmax(scores, dim=-1)
        print('Weights : [Softmax(scores)] : ',weights.shape)
        context = torch.bmm(weights, keys)
        print('Context : [batched-Matrix Mutliplication(weights, keys)] :',context.shape)
        print('Attn Weight shape :',weights.shape)
        print(20*'-',' Attention Inference End ',20*'-')
        return context, weights
# Attention 2

class DotProductAttentionLinear(nn.Module):
    def __init__(self, hidden_size):
        super(DotProductAttentionLinear, self).__init__()

        # Linear transformations for query, key, and value
        self.query_linear = nn.Linear(hidden_size, hidden_size)
        self.key_linear = nn.Linear(hidden_size, hidden_size)
        self.value_linear = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, keys, values, mask=None):
        # Linear transformations of query, key, and value
        query = self.query_linear(query)
        keys = self.key_linear(keys)
        values = self.value_linear(values)
        # Calculate dot products
        dot_products = torch.bmm(query, keys.transpose(1, 2))

        # Apply mask (if provided)
        if mask is not None:
            dot_products.masked_fill_(mask, float('-inf'))

        # Apply softmax to calculate attention weights
        attention_weights = torch.nn.functional.softmax(dot_products, dim=-1)

        # Calculate the weighted sum of values
        context = torch.bmm(attention_weights, values)

        return context, attention_weights
#Attention 3
class SelfAttention(nn.Module):
    def __init__(self, hidden_size):
        super(SelfAttention, self).__init__()

        self.hidden_size =hidden_size

        # Linear transformations for queries, keys, and values
        self.q_linear = nn.Linear(hidden_size, hidden_size)
        self.k_linear = nn.Linear(hidden_size, hidden_size)
        self.v_linear = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, key, value, mask=None):
        # Linear transformations
        Q = self.q_linear(query)
        K = self.k_linear(key)
        V = self.v_linear(value)

        # Scaled Dot-Product Attention
        scores = torch.matmul(Q, K.permute(0, 2, 1)) / torch.sqrt(torch.tensor(self.hidden_size, dtype=torch.float32))


        if mask is not None:
            scores = scores.masked_fill(mask == 0, float("-inf"))

        attention = F.softmax(scores, dim=-1)
        output = torch.matmul(attention, V)

        return output, attention
#Attention 4
class LocationBasedAttention(nn.Module):
    def __init__(self, hidden_size):
        super(LocationBasedAttention, self).__init__()
        self.Wq = nn.Linear(hidden_size, hidden_size)
        self.Wk = nn.Linear(hidden_size, hidden_size)
        self.Wv = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, keys):
        # Linear transformations
        q = self.Wq(query)
        k = self.Wk(keys)
        v = self.Wv(keys)

        # Calculate attention scores (typically position-based)
        scores = torch.matmul(q, k.transpose(1, 2))

        # Apply softmax to obtain attention weights
        weights = torch.nn.functional.softmax(scores, dim=-1)

        # Compute the context vector based on attention weights
        context = torch.matmul(weights, v)

        return context, weights
#attention 5
class GlobalAttention(nn.Module):
    def __init__(self, hidden_size):
        super(GlobalAttention, self).__init__()
        self.Wq = nn.Linear(hidden_size, hidden_size)
        self.Wk = nn.Linear(hidden_size, hidden_size)
        self.Wv = nn.Linear(hidden_size, hidden_size)

    def forward(self, query, keys):
        # Linear transformations
        q = self.Wq(query)
        k = self.Wk(keys)
        v = self.Wv(keys)

        # Calculate attention scores (global attention)
        scores = torch.matmul(q, k.transpose(1, 2))

        # Apply softmax to obtain attention weights
        weights = torch.nn.functional.softmax(scores, dim=-1)

        # Compute the context vector based on attention weights
        context = torch.matmul(weights, v)

        return context, weights

In [16]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,attention, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention_name=attention
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)
        if self.attention_name=='BahdanauAttention':
            self.attention = BahdanauAttention(hidden_size)
        elif self.attention_name=='DotProductAttentionLinear':
            self.attention=DotProductAttentionLinear(hidden_size)
        elif self.attention_name =='SelfAttention':
            self.attention=SelfAttention(hidden_size)
        elif self.attention_name == 'LocationBasedAttention':
            self.attention=LocationBasedAttention(hidden_size)
        elif self.attention_name == 'GlobalAttention':
            self.attention=GlobalAttention(hidden_size)
        else:
            print('Error choose Correct Attention')
            
            

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))
        
        query = hidden.permute(1, 0, 2)
        
        if self.attention_name=='BahdanauAttention':
            context, attn_weights = self.attention(query, encoder_outputs)
        elif self.attention_name=='DotProductAttentionLinear':
            context, attn_weights = self.attention(query, encoder_outputs,encoder_outputs)
        elif self.attention_name=='SelfAttention':
            context, attn_weights = self.attention(query, encoder_outputs,encoder_outputs)
        elif self.attention_name== 'LocationBasedAttention':
            context, attn_weights = self.attention(query, encoder_outputs)
        elif self.attention_name == 'GlobalAttention':
            context, attn_weights = self.attention(query, encoder_outputs)
        else:
            print('Error choose Correct Attention')
            
        input_gru = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights
    def forward_step_attention_inference(self, input, hidden, encoder_outputs):
        print(20*'-',' Forward Step Inference start',20*'-')
        embedded =  self.dropout(self.embedding(input))
        print('Embedding shape :',embedded.shape)
        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention.forward_inference(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)
        print('Input to the Gru Concatenation of the embedded and context on dim 2 :',input_gru.shape)
        output, hidden = self.gru(input_gru, hidden)
        print('Output :',output.shape)
        print('Hidden :',hidden.shape)
        output = self.out(output)
        print('Final Linear Ouptut .shape ',output.shape)
        print(20*'-',' Forward Step Inference End',20*'-')
        return output, hidden, attn_weights
    def inference_shape(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []
        print(50*'-','Forward Method begin',50*'-')
        print('Batch Size : ',batch_size)
        print('decoder Input shape:',decoder_input.shape)
        #print('decoder Input : ',decoder_input)
        print('decoder Hidden :',decoder_hidden.shape)
        print(20*'-','Decoder Mapping Start ',20*'-')
        for i in range(MAX_LENGTH):
            print(f'Iteration : [{MAX_LENGTH+1}/{i+1}]  --->  Iteration : {i+1}')
            decoder_output, decoder_hidden, attn_weights = self.forward_step_attention_inference(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)
            print(f'Iteration {i+1} | Deocder output shape :{decoder_output.shape}: Lenght of List of Decoder Output : ',len(decoder_outputs))
            print(f'Iteration {i+1} | Attention Weights : ',attn_weights.shape)
            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
                print('Teacher Forcing | Feed the target as the next input :',target_tensor[:, i].unsqueeze(1).shape)
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input
                print('Without teacher forcing: use its own predictions as the next input : ',decoder_output.topk(1).shape)
            print(20*'-','Decoder Mapping End',20*'-')
        print('*********Forward Step Function******')
        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        print('Final Decoder output concatenation : ',decoder_outputs.shape)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        print('Softmax Applying Decoder outputs : ',decoder_outputs.shape)
        attentions = torch.cat(attentions, dim=1)
        print('Attention concatenation to dim : 1',attentions.shape)
        print(50*'-','Forward Method End',50*'-')
        return decoder_outputs, decoder_hidden, attentions

In [17]:
def indexesFromSentence(lang, sentence):
    words=sentence.split(' ')
    return [lang.word2index.get(word,UNK_token) for word in words]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData(data,'eng', 'Hinglish', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader


In [18]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion,inference):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()
        
        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()
        
    return total_loss / len(dataloader)
def train_epoch_inference(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion,inference):
    total_loss = 0
    if inference:
            for data in dataloader:
                input_tensor, target_tensor = data

                encoder_optimizer.zero_grad()
                decoder_optimizer.zero_grad()

                encoder_outputs, encoder_hidden = encoder(input_tensor)
                decoder_outputs, _, _ = decoder.inference_shape(encoder_outputs, encoder_hidden, target_tensor)

                loss = criterion(
                    decoder_outputs.view(-1, decoder_outputs.size(-1)),
                    target_tensor.view(-1)
                )
                loss.backward()

                encoder_optimizer.step()
                decoder_optimizer.step()

                total_loss += loss.item()
                print('Input Shape : ',input_tensor.shape)
                print('Output Shape :',target_tensor.shape)
                print('Encoder Outputs : ',encoder_outputs.shape)
                print('Encoder Hidden :',encoder_hidden.shape)
                print('Decoder _outputs :',decoder_outputs.shape)
                print('Loss->Decoder.shape :',decoder_outputs.view(-1, decoder_outputs.size(-1)).shape)
                print('Loss-> Target shape :',target_tensor.view(-1).shape)
                print('Loss : ',loss.item())
                break
    return total_loss / len(dataloader)

In [19]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [20]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100,inference=False):
    if inference:
        print('Inference Start :Set to Epoch : 1 ')
        print(time.time())
        n_epochs=1
        
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    
    for epoch in range(1, n_epochs + 1):
        if inference :
            train_epoch_inference(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,inference)
            print('Inference End ')
            return 
        else:
            loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,inference)
            print_loss_total += loss
            plot_loss_total += loss

            if epoch % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print(f'Time : -{timeSince(start, epoch / n_epochs)} | Epoch : {epoch} | Complete : {epoch / n_epochs * 100}%  | Average Loss :{print_loss_avg}')
                

            if epoch % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
    return plot_losses

In [21]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [22]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn
def evaluateRandomly(encoder, decoder, n=100):
    predictions = {
        'actual': [],
        'predict': [],
        'target': []
    }

    for i in range(n):
        pair = random.choice(pairs)

        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)

        predictions['actual'].append(pair[0])
        predictions['target'].append(pair[1])
        predictions['predict'].append(output_sentence)
        if i<=5:
            print('>', pair[0])
            print('=', pair[1])
            print('<', output_sentence)
            print('')
    return predictions


In [24]:
hidden_size = 128
batch_size = 32
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)
print('Initializing Attentions ......')
Atten_losses={}
attentions={'bha':'BahdanauAttention','dpa':'DotProductAttentionLinear',
            'sa':'SelfAttention','lba':'LocationBasedAttention',
            'gba':'GlobalAttention'}
map_prediction={}
for keys,value in attentions.items():
    print(f'Training with : {value}')
    encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size, output_lang.n_words,attention=value).to(device)
    print('Training Start..')
    losses=train(train_dataloader, encoder, decoder, 50, print_every=5, plot_every=5,inference=False)
    Atten_losses[keys]=losses
    print('Training End..')
    print(f'{value} - Evaluation ....')
    encoder.eval()
    decoder.eval()
    predictions=evaluateRandomly(encoder, decoder)
    map_prediction[value]=predictions
    print(f'{value} Evaluation End ...')
    print()

Reading lines...
Preparing Vocabulary.......
Read 50000 sentence pairs
Trimmed to 27839 sentence pairs
Counting words
Counted words :
eng 5971
Hinglish 6393
Initializing Attentions ......
Training with : BahdanauAttention
Training Start..
Time : -1m 22s (- 12m 25s) | Epoch : 5 | Complete : 10.0%  | Average Loss :1.5017988410557823
Time : -2m 43s (- 10m 52s) | Epoch : 10 | Complete : 20.0%  | Average Loss :0.5245570382578619
Time : -4m 2s (- 9m 26s) | Epoch : 15 | Complete : 30.0%  | Average Loss :0.2804525253587756
Time : -5m 22s (- 8m 4s) | Epoch : 20 | Complete : 40.0%  | Average Loss :0.1875778607373265
Time : -6m 43s (- 6m 43s) | Epoch : 25 | Complete : 50.0%  | Average Loss :0.1412806620226167
Time : -8m 2s (- 5m 21s) | Epoch : 30 | Complete : 60.0%  | Average Loss :0.11385311123369068
Time : -9m 21s (- 4m 0s) | Epoch : 35 | Complete : 70.0%  | Average Loss :0.09637685755674524
Time : -10m 41s (- 2m 40s) | Epoch : 40 | Complete : 80.0%  | Average Loss :0.08406569045293948
Time : -

In [25]:
loss=pd.DataFrame(Atten_losses,columns=Atten_losses.keys())
loss.to_csv('Attention_losses.csv',index=None)
loss.head()

,bha,dpa,sa,lba,gba
0,1.501799,1.649955,1.575552,1.713533,1.692379
1,0.524557,0.717538,0.581176,0.775331,0.782499
2,0.280453,0.442303,0.334230,0.494137,0.534645
3,0.187578,0.306856,0.233934,0.351893,0.410286
4,0.141281,0.234432,0.182787,0.275239,0.340804


In [28]:
blu_score={}
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu
# Create a dictionary to store the data frames
attention_data_frames = {}
# Assuming map_prediction is your dictionary
for attention, predictions in map_prediction.items():
    df = pd.DataFrame(predictions)
    # Store the data frame in the dictionary with a short name
    attention_data_frames[attention] = df
    
for key,values in attention_data_frames.items():
    print('Attention : ',key)
    references = values['target']
    hypotheses = values['predict']
    # Calculate BLEU scores for the entire batch
    bleu_scores = corpus_bleu(references, hypotheses)
    # Print the BLEU scores
    print("BLEU Scores: ", bleu_scores)
    blu_score[keys]=bleu_scores
    print()

Attention :  BahdanauAttention
BLEU Scores:  0.7684297726002483

Attention :  DotProductAttentionLinear
BLEU Scores:  0.7665481564327498

Attention :  SelfAttention
BLEU Scores:  0.7617910632731427

Attention :  LocationBasedAttention
BLEU Scores:  0.7703729490539172

Attention :  GlobalAttention
BLEU Scores:  0.7709167710280911



In [34]:
import matplotlib.pyplot as plt
# Extract the language pairs and scores as separate lists
attentions = list(blu_score.keys())
scores = list(blu_score.values())

# Create the bar chart
plt.bar(attentions, scores)

# Add labels and a title
plt.xlabel('Attentions')
plt.ylabel('BLEU Score')
plt.title('BLEU Scores for Different Attentions')
# Show the plot
plt.show()

# Training Model With Attention (that has best Bleu Score)

In [35]:
hidden_size = 128
batch_size = 32
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)
print('Initializing Attentions ......')
Atten_losses={}
attentions={'gba':'GlobalAttention'}
map_prediction={}
for keys,value in attentions.items():
    print(f'Training with : {value}')
    encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size, output_lang.n_words,attention=value).to(device)
    print('Training Start..')
    losses=train(train_dataloader, encoder, decoder, 70, print_every=5, plot_every=5,inference=False)
    Atten_losses[keys]=losses
    print('Training End..')
    print(f'{value} - Evaluation ....')
    encoder.eval()
    decoder.eval()
    predictions=evaluateRandomly(encoder, decoder)
    map_prediction[value]=predictions
    print(f'{value} Evaluation End ...')
    print('Saving the Model')
    # Save the encoder model
    torch.save(encoder.state_dict(), 'encoder_model.pt')
    # Save the decoder model
    torch.save(decoder.state_dict(), 'decoder_model.pt')
    print('Model is Saved')
    print()

Reading lines...
Preparing Vocabulary.......
Read 50000 sentence pairs
Trimmed to 27839 sentence pairs
Counting words
Counted words :
eng 5971
Hinglish 6393
Initializing Attentions ......
Training with : GlobalAttention
Training Start..
Time : -1m 30s (- 19m 31s) | Epoch : 5 | Complete : 7.142857142857142%  | Average Loss :1.6962527414025932
Time : -2m 59s (- 17m 59s) | Epoch : 10 | Complete : 14.285714285714285%  | Average Loss :0.7954896988128793
Time : -4m 29s (- 16m 28s) | Epoch : 15 | Complete : 21.428571428571427%  | Average Loss :0.5307124808533439
Time : -5m 58s (- 14m 56s) | Epoch : 20 | Complete : 28.57142857142857%  | Average Loss :0.4200999793924134
Time : -7m 27s (- 13m 26s) | Epoch : 25 | Complete : 35.714285714285715%  | Average Loss :0.33933049223203765
Time : -8m 57s (- 11m 56s) | Epoch : 30 | Complete : 42.857142857142854%  | Average Loss :0.30316461343361045
Time : -10m 26s (- 10m 26s) | Epoch : 35 | Complete : 50.0%  | Average Loss :0.2780036887166829
Time : -11m 55